# PARSING PDF FILE TO EXTRACT INFORMATION ABOUT VARIABLES AND INSERT INTO METADATA TABLE

In [69]:
import pandas as pd

In [70]:
import os

In [71]:
import openpyxl

In [72]:
import numpy as np
import cvxpy as cp

In [73]:
file_path = "../../GIT/demo-Turin-EuSilc/02_DataSet/" # Set the directory of the EU-SULC files

In [74]:
df = pd.read_excel(os.path.join(file_path,"Calibration_variables.xlsx"))

In [75]:
df.head

<bound method NDFrame.head of               id  classe_eta  genere
0     1571500001           3       2
1     1571500002           5       2
2     1571560001           4       1
3     1571560002           4       2
4     1571680001           4       2
...          ...         ...     ...
4271  2425930002           5       1
4272  2425960001           3       2
4273  2425960002           1       2
4274  2425960003           1       1
4275  2425960004           4       1

[4276 rows x 3 columns]>

In [76]:
np.random.seed(42)
n = len(df)
print(n)

4276


In [77]:
df['design_weight'] = 1.0

In [78]:
df['classe_eta'] = df['classe_eta'].astype(str)
df['genere'] = df['genere'].astype(str)
df['group'] = df['classe_eta'] + "_" + df['genere']

In [79]:
X = pd.get_dummies(df['group'])
print(X)

      1_1  1_2  2_1  2_2  3_1  3_2  4_1  4_2  5_1  5_2
0       0    0    0    0    0    1    0    0    0    0
1       0    0    0    0    0    0    0    0    0    1
2       0    0    0    0    0    0    1    0    0    0
3       0    0    0    0    0    0    0    1    0    0
4       0    0    0    0    0    0    0    1    0    0
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
4271    0    0    0    0    0    0    0    0    1    0
4272    0    0    0    0    0    1    0    0    0    0
4273    0    1    0    0    0    0    0    0    0    0
4274    1    0    0    0    0    0    0    0    0    0
4275    0    0    0    0    0    0    1    0    0    0

[4276 rows x 10 columns]


In [80]:
calibration_vars = X.columns.tolist()

print(calibration_vars)

['1_1', '1_2', '2_1', '2_2', '3_1', '3_2', '4_1', '4_2', '5_1', '5_2']


In [81]:
population_totals = {
    '1_1': 90,
    '1_2': 90,
    '2_1': 100,
    '2_2': 130,
    '3_1': 240,
    '3_2': 250,
    '4_1': 325,
    '4_2': 335,
    '5_1': 615,
    '5_2': 545
}
t = np.array([population_totals[var] for var in calibration_vars])
print(t)

[ 90  90 100 130 240 250 325 335 615 545]


In [82]:
# Logit bounds: 0.01 ≤ g ≤ 4
lower = 0.01
upper = 4

In [83]:
g = cp.Variable(n)
print(g)

var163


In [84]:
A = X.values  # (n x k)
print(A)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [85]:
w = df['design_weight'].values  # (n, )
print(w)

[1. 1. 1. ... 1. 1. 1.]


In [86]:
constraints = [
    g >= lower,
    g <= upper,
    A.T @ cp.multiply(g, w) == t
]
print(constraints)

[Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Variable((4276,), var163)), Equality(Expression(AFFINE, UNKNOWN, (10,)), Constant(CONSTANT, NONNEGATIVE, (10,)))]


In [87]:
objective = cp.Minimize(cp.sum(cp.kl_div(g, np.ones(n))))

In [88]:
problem = cp.Problem(objective, constraints)
problem.solve(verbose=True)

                                     CVXPY                                     
                                     v1.6.4                                    
(CVXPY) Apr 13 07:41:37 PM: Your problem has 4276 variables, 8562 constraints, and 0 parameters.
(CVXPY) Apr 13 07:41:37 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 13 07:41:37 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 13 07:41:37 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 13 07:41:37 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 13 07:41:37 PM: Compiling problem (target solver=CLARAB

454.7520129837677

In [89]:
df['calibrated_weight'] = g.value * df['design_weight']

In [90]:
print("\nConfronto tra margini campionari e obiettivi:")
sample_totals = A.T @ w
calibrated_totals = A.T @ df['calibrated_weight'].values
results = pd.DataFrame({
    'Group': calibration_vars,
    'Target': t,
    'Original': sample_totals,
    'Calibrated': calibrated_totals
})
print(results)



Confronto tra margini campionari e obiettivi:
  Group  Target  Original  Calibrated
0   1_1      90     228.0   90.000000
1   1_2      90     236.0   90.000000
2   2_1     100     122.0  100.000000
3   2_2     130     122.0  130.000000
4   3_1     240     438.0  240.000000
5   3_2     250     430.0  250.000000
6   4_1     325     632.0  325.000000
7   4_2     335     668.0  335.000000
8   5_1     615     594.0  614.999999
9   5_2     545     806.0  544.999999


In [91]:
df.head

<bound method NDFrame.head of               id classe_eta genere  design_weight group  calibrated_weight
0     1571500001          3      2            1.0   3_2           0.581395
1     1571500002          5      2            1.0   5_2           0.676179
2     1571560001          4      1            1.0   4_1           0.514241
3     1571560002          4      2            1.0   4_2           0.501497
4     1571680001          4      2            1.0   4_2           0.501497
...          ...        ...    ...            ...   ...                ...
4271  2425930002          5      1            1.0   5_1           1.035354
4272  2425960001          3      2            1.0   3_2           0.581395
4273  2425960002          1      2            1.0   1_2           0.381356
4274  2425960003          1      1            1.0   1_1           0.394741
4275  2425960004          4      1            1.0   4_1           0.514241

[4276 rows x 6 columns]>